In [ ]:
%matplotlib inline
import vdmlab as vdm
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from core import assign_label, Rat, combine_rats
from plotting import plot_behavior

In [ ]:
home = os.path.expanduser("~")
root = os.path.join(home, 'code', 'emi_biconditional')
data_filepath = os.path.join(root, 'cache', 'data')
split_data_filepath = os.path.join(root, 'cache', 'other')
output_filepath = os.path.join(root, 'plots')

In [ ]:
sessions = []
for file in sorted(os.listdir(data_filepath)):
    if file[0] == '!':
        sessions.append(os.path.join(data_filepath, file))

rats = ['1', '2', '3', '4', '5', '6', '7', '8']

data = dict()
for rat in rats:
    data[rat] = Rat(rat)

broken_a = os.path.join(split_data_filepath, '!2016-10-19a1')
broken_b = os.path.join(split_data_filepath, '!2016-10-19a2')
rats_data_a = vdm.load_medpc(broken_a, assign_label)
rats_data_b = vdm.load_medpc(broken_b, assign_label)
for rat in rats_data_a:
    for key in rats_data_a[rat]:
        rats_data_b[rat][key].join(rats_data_a[rat][key])

for rat in rats:
    data[rat].add_session(**rats_data_b[rat])

for session in sessions:
    rats_data = vdm.load_medpc(os.path.join(data_filepath, session), assign_label)

    for rat in rats:
        data[rat].add_session(**rats_data[rat])

n_sessions = len(data['1'].sessions)
only_sound = False

df = combine_rats(data, rats, n_sessions, only_sound)

In [ ]:
def fix_missing_trials(df):
    nan_idx = np.where(np.isnan(df['value']))[0]
    for idx in nan_idx:
        row = df.loc[idx]
        value = df.loc[(df['rat'] == row['rat']) & 
                       (df['session'] == row['session']) & 
                       (df['condition'] == row['condition']) & 
                       (df['measure'] == row['measure'])].mean()['value']

        df.set_value(idx, 'value', value)

In [ ]:
fix_missing_trials(df)

In [ ]:
np.sum(np.isnan(df['value']))

In [ ]:
plot_behavior(df, ['1'], only_sound=False, by_outcome=True)